In [23]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent.parent))

In [24]:
import time
import triton
import triton.language as tl
import torch
import torch.nn.functional as F
import pandas as pd
from conv_gemm.triton_kernels.fp16.img2col_kernel import img2col_kernel

In [25]:
def img2col_fp16(
    x,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    BLOCK_M,
    BLOCK_K,
    num_warps,
    num_stages,
):
    assert x.is_cuda
    assert x.dtype in (torch.float16, torch.float32)

    N, Cin, H, W = x.shape

    Ho = (H + 2*Ph - Dh*(Kh - 1) - 1)//Sh + 1
    Wo = (W + 2*Pw - Dw*(Kw - 1) - 1)//Sw + 1

    M = N * Ho * Wo
    K = Cin * Kh * Kw

    cols = torch.empty((M, K), dtype=x.dtype, device=x.device)
    sN, sC, sH, sW = x.stride()

    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(K, BLOCK_K))

    img2col_kernel[grid](
        x, cols,
        N, Cin, H, W,
        Kh, Kw, Sh, Sw, Ph, Pw, Dh, Dw,
        Ho, Wo,
        sN, sC, sH, sW,
        K,
        BLOCK_M=BLOCK_M,
        BLOCK_K=BLOCK_K,
        CAST_FP16=(x.dtype == torch.float16),
        num_warps=num_warps,
        num_stages=num_stages,
    )

    return cols, (Ho, Wo)


In [26]:
@torch.no_grad()
def bench_once_img2col_fp16_vs_unfold(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    BLOCK_M,
    BLOCK_K,
    num_warps,
    num_stages,
    iters=100,
    device="cuda",
):

    x = torch.randn((N, Cin, H, W), device=device, dtype=torch.float16)
    x_f16 = x.float().half()

    Ho = (H + 2 * Ph - Dh * (Kh - 1) - 1) // Sh + 1
    Wo = (W + 2 * Pw - Dw * (Kw - 1) - 1) // Sw + 1

    K = Cin * Kh * Kw
    M = N * Ho * Wo

    # TORCH 
    def _call_torch():
        return F.unfold(
            x_f16,
            kernel_size=(Kh, Kw),
            dilation=(Dh, Dw),
            padding=(Ph, Pw),
            stride=(Sh, Sw),
        )

    # warmup
    for _ in range(5):
        _call_torch()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        unf = _call_torch()
    torch.cuda.synchronize()
    t_torch = (time.perf_counter() - t0) / iters

    # torch output → [M, K]
    unf_M_K = unf.permute(0, 2, 1).contiguous().view(M, K)

    # ----- TRITON FP16 IMG2COL -----
    def _call_triton():
        cols, (Ho2, Wo2) = img2col_fp16(
            x,
            Kh, Kw,
            Sh, Sw,
            Ph, Pw,
            Dh, Dw,
            BLOCK_M=BLOCK_M,
            BLOCK_K=BLOCK_K,
            num_warps=num_warps,
            num_stages=num_stages,
        )
        assert Ho2 == Ho and Wo2 == Wo
        return cols

    for _ in range(5):
        _call_triton()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        cols = _call_triton()
    torch.cuda.synchronize()
    t_triton = (time.perf_counter() - t0) / iters

    # Error Triton INT32 vs Torch FP16
    cols_f32 = cols.float()
    unf_f32  = unf_M_K.float()

    diff = (cols_f32 - unf_f32).abs()
    max_abs_err16 = diff.max().item()
    mean_abs_err16 = diff.mean().item()
    
    # bandwidth (FP16 → 2 bytes)
    bytes_moved = 2.0 * M * K * 2   # load + store * fp16(2 bytes)
    bw_triton = bytes_moved / t_triton / 1e9
    bw_torch  = bytes_moved / t_torch / 1e9

    return {
        "M": M, "K": K, "N": N,
        "BLOCK_M": BLOCK_M,
        "BLOCK_N": 0,
        "BLOCK_K": BLOCK_K,
        "num_warps": num_warps,
        "num_stages": num_stages,
        "t_triton_ms": t_triton * 1e3,
        "t_torch16_ms": t_torch * 1e3,
        "speed_vs_torch16": t_torch / t_triton,
        "bw_triton_GBs": bw_triton,
        "bw_torch16_GBs": bw_torch,
        "max_abs_err": max_abs_err16,
        "mean_abs_err": mean_abs_err16,
    }

In [27]:
@torch.no_grad()
def tune_img2col_fp16_tiles_for_shape(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(1, 2, 4, 8),
    stages=(2, 3, 4),
    iters=200,
    device="cuda",
):
    records = []
    for BM in blocks_M:
        for BK in blocks_K:
            for W in warps:
                for S in stages:
                    try:
                        rec = bench_once_img2col_fp16_vs_unfold(
                            N, Cin, H, W,
                            Kh, Kw,
                            Sh, Sw,
                            Ph, Pw,
                            Dh, Dw,
                            BLOCK_M=BM,
                            BLOCK_K=BK,
                            num_warps=W,
                            num_stages=S,
                            iters=iters,
                            device=device,
                        )
                    except RuntimeError as e:
                        print(f"[SKIP] BM={BM}, BK={BK}, W={W}, S={S}: {e}")
                        continue

                    print(
                        f"BM={BM}, BK={BK}, W={W}, S={S}: "
                        f"t_triton={rec['t_triton_ms']:.3f} ms, "
                        f"speed_vs_torch={rec['speed_vs_torch16']:.3f}x, "
                    )
                    records.append(rec)

    if not records:
        raise RuntimeError("No valid tile configs found for this IMG2COL shape")

    df = pd.DataFrame(records)
    return df


In [28]:
df_fp16_i2c = tune_img2col_fp16_tiles_for_shape(
    N=16, Cin=1, H=256, W=256,
    Kh=11, Kw=11,
    Sh=1, Sw=1,
    Ph=5, Pw=5,
    Dh=1, Dw=1,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(2, 4, 8, 16),
    stages=(2, 3),
    iters=200,
    device="cuda",
)


BM=32, BK=32, W=2, S=2: t_triton=0.037 ms, speed_vs_torch=6.386x, 
BM=32, BK=32, W=2, S=3: t_triton=0.037 ms, speed_vs_torch=6.055x, 
BM=32, BK=32, W=4, S=2: t_triton=0.038 ms, speed_vs_torch=10.270x, 
BM=32, BK=32, W=4, S=3: t_triton=0.038 ms, speed_vs_torch=9.993x, 
BM=32, BK=32, W=8, S=2: t_triton=0.047 ms, speed_vs_torch=8.045x, 
BM=32, BK=32, W=8, S=3: t_triton=0.047 ms, speed_vs_torch=9.271x, 
BM=32, BK=32, W=16, S=2: t_triton=0.115 ms, speed_vs_torch=3.351x, 
BM=32, BK=32, W=16, S=3: t_triton=0.115 ms, speed_vs_torch=3.218x, 
BM=32, BK=64, W=2, S=2: t_triton=0.038 ms, speed_vs_torch=5.456x, 
BM=32, BK=64, W=2, S=3: t_triton=0.045 ms, speed_vs_torch=4.514x, 
BM=32, BK=64, W=4, S=2: t_triton=0.038 ms, speed_vs_torch=9.979x, 
BM=32, BK=64, W=4, S=3: t_triton=0.038 ms, speed_vs_torch=11.180x, 
BM=32, BK=64, W=8, S=2: t_triton=0.043 ms, speed_vs_torch=8.701x, 
BM=32, BK=64, W=8, S=3: t_triton=0.043 ms, speed_vs_torch=8.545x, 
BM=32, BK=64, W=16, S=2: t_triton=0.093 ms, speed_vs_torch

In [29]:
cols = [
    "BLOCK_M", "BLOCK_K",
    "num_warps", "num_stages",
    "t_triton_ms", "t_torch16_ms",
    "speed_vs_torch16",
     "mean_abs_err"
]

df_i2c_filtered = df_fp16_i2c[cols].sort_values("speed_vs_torch16", ascending=False).head(5).reset_index(drop=True)
df_i2c_filtered

,BLOCK_M,BLOCK_K,num_warps,num_stages,t_triton_ms,t_torch16_ms,speed_vs_torch16,mean_abs_err
0,128,64,4,3,0.037922,0.429328,11.321315,0.0
1,64,64,4,3,0.037786,0.424942,11.246101,0.0
2,32,64,4,3,0.037562,0.419957,11.180372,0.0
3,128,32,8,3,0.040342,0.439529,10.894965,0.0
4,32,128,4,3,0.036904,0.393694,10.668010,0.0


In [9]:
FP16_IMG2COL_BEST_BLOCK_M = 32
FP16_IMG2COL_BEST_BLOCK_N = 0
FP16_IMG2COL_BEST_BLOCK_K = 64
FP16_IMG2COL_BEST_WARPS   = 2
FP16_IMG2COL_BEST_STAGES  = 2